In [1]:
from __future__ import print_function

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn

In [3]:
import torchvision
import torchvision.transforms as transforms

In [4]:
import os
import sys
import time
import argparse
import numpy as np

In [5]:
from torch.autograd import Variable

In [6]:
import config as cf
from dataset import dataset as dset

In [7]:
use_cuda = torch.cuda.is_available()

In [8]:
best_acc = 0

In [9]:
# Data Uplaod
print('\n[Phase 1] : Data Preparation')
trainloader, testloader, num_classes, inputs = dset('mnist')
print ('Number of classes: {} \n Number of input channels: {}'.format(num_classes,inputs))


[Phase 1] : Data Preparation
| Preparing MNIST dataset...
| Number of classes: 10 
 Number of input channels: 1


In [74]:
class TwoLayerLinear(nn.Module):

    def __init__(self, num_classes=num_classes, inputs=inputs):
        super(TwoLayerLinear, self).__init__()
        self.fc1 = nn.Linear(inputs, 256) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(256, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [75]:
net = TwoLayerLinear(num_classes = num_classes,inputs=32)
file_name = 'TwoLayerLinear'

In [76]:
if use_cuda:
    net.cuda()
    net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True

In [77]:
criterion = nn.CrossEntropyLoss()

In [78]:
def train(epoch):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    optimizer = optim.Adam(net.parameters(), lr=cf.lr, weight_decay=5e-4)

    print('\n=> Training Epoch #%d, LR=%.4f' %(epoch, cf.lr))
    for batch_idx, (inputs_value, targets) in enumerate(trainloader):
        if use_cuda:
            inputs_value, targets = inputs_value.cuda(), targets.cuda() # GPU settings
        optimizer.zero_grad()
        inputs_value, targets = Variable(inputs_value), Variable(targets)
        outputs = net(inputs_value)               # Forward Propagation
        loss = criterion(outputs, targets)  # Loss
        loss.backward()  # Backward Propagation
        optimizer.step() # Optimizer update

        train_loss += loss.data[0]
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        print ('| Epoch [%3d/%3d] Iter[%3d/%3d]\t\tLoss: %.4f Acc@1: %.3f%%'
                %(epoch, cf.num_epochs, batch_idx+1,
                    (len(trainset)//cf.batch_size)+1, loss.data[0], 100.*correct/total))

In [79]:
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs_value, targets) in enumerate(testloader):
        if use_cuda:
            inputs_value, targets = inputs_value.cuda(), targets.cuda()
        with torch.no_grad():
            inputs_value, targets = Variable(inputs_value), Variable(targets)
        outputs = net(inputs_value)
        loss = criterion(outputs, targets)

        test_loss += loss.data[0]
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

    # Save checkpoint when best model
    acc = 100.*correct/total
    print("\n| Validation Epoch #%d\t\t\tLoss: %.4f Acc@1: %.2f%%" %(epoch, loss.data[0], acc))
    
    if acc > best_acc:
        best_acc = acc

In [80]:
def get_hms(seconds):
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)

    return h, m, s

In [81]:
elapsed_time = 0
for epoch in range(cf.start_epoch, cf.start_epoch+cf.num_epochs):
    start_time = time.time()

    train(epoch)
    test(epoch)

    epoch_time = time.time() - start_time
    elapsed_time += epoch_time
    print('| Elapsed time : %d:%02d:%02d'  %(get_hms(elapsed_time)))


=> Training Epoch #1, LR=0.0010


Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f6162e95160>>
Traceback (most recent call last):
  File "/home/insideout/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 349, in __del__
    self._shutdown_workers()
  File "/home/insideout/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 328, in _shutdown_workers
    self.worker_result_queue.get()
  File "/home/insideout/anaconda3/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/home/insideout/anaconda3/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/home/insideout/anaconda3/lib/python3.6/multiprocessing/resource_sharer.py", line 58, in detach
    return reduction.recv_handle(conn)
  File "/home/insideout/anaconda3/lib/python3.6/multiprocessing/reduction.py", line 182, in recv_h

RuntimeError: invalid argument 1: only batches of spatial targets supported (3D tensors) but got targets of dimension: 1 at /opt/conda/conda-bld/pytorch_1525909934016/work/aten/src/THCUNN/generic/SpatialClassNLLCriterion.cu:14

In [ ]:
print('\n[Phase 4] : Testing model')
print('* Test results : Acc@1 = %.2f%%' %(best_acc))

In [ ]:
dataset = 'cifar10'

In [ ]:
elapsed_time = 0
for epoch in range(start_epoch, start_epoch+num_epochs):
    start_time = time.time()

    train(epoch)
    test(epoch)

    epoch_time = time.time() - start_time
    elapsed_time += epoch_time
    print('| Elapsed time : %d:%02d:%02d'  %(get_hms(elapsed_time)))

In [ ]:
To use:
    %time
    %prun
    Add progress bar : https://github.com/fastai/fastprogress
            